# Find your Dog's Family!

In [16]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [17]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [18]:
key = os.environ.get('AZURE_SEARCH_KEY', '<KEY>')

In [19]:
results = search_images_bing(key, 'german shepherd dogs')
ims = results.attrgot('content_url')

In [20]:
dest = 'images/example.jpg'
download_url(ims[1], dest)

In [21]:
dog_types = ['german shepherd', 'black', 'labrador']
path = Path('dogs')

In [22]:
if not path.exists():
    path.mkdir()
    for t in dog_types:
        dest = (path/t)
        print(dest)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, '{} dog'.format(t))
        download_images(dest, urls=results.attrgot('content_url'))
    

In [23]:
fns= get_image_files(path)

In [24]:
##finding out the no. of failed images
failed = verify_images(fns)

In [25]:
##removing all the failed images using unlink
failed.map(Path.unlink);

In [26]:
##
dogs = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        splitter=RandomSplitter(valid_pct=0.2, seed=41),
        get_y=parent_label,
        item_tfms=Resize(128)
        )

In [27]:
dls = dogs.dataloaders(path)

In [28]:
# dls.valid.show_batch(max_n = 4, nrows=1)

In [29]:
dogs = dogs.new(item_tfms=RandomResizedCrop(224, min_scale=0.5),
               batch_tfms=aug_transforms())

dls = dogs.dataloaders(path)

In [30]:
learn.export()

In [31]:
path = Path()

In [32]:
learn_inf = load_learner(path/'export.pkl')

In [33]:
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Classify')
out_pl = widgets.Output()
lbl_pred = widgets.Label()

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    
btn_run.on_click(on_click_classify)

In [34]:
VBox([widgets.Label('Select your dog!'),
      btn_upload, btn_run, out_pl, lbl_pred])